In [36]:
import cvxpy as cp
import numpy as np
import networkx as nx


inputnodes = [0,5]
inputedges = [(0,2),(4,5),(1,3),(0,4),(4,1),(2,5),(5,3)]

non_member = [4,5]
source = [0,1]
recievers = [[3],[2]]

sessions = [0,1]



preG = nx.Graph()
preG.add_nodes_from(inputnodes)
preG.add_edges_from(inputedges)
directE = []
for n, nbrs in preG.adj.items():
    for nbr, eattr in nbrs.items():
        directE.append((n,nbr))


G = nx.DiGraph()
G.add_nodes_from(inputnodes)
G.add_edges_from(directE)

undirected_edges = set(inputedges)

vertices = set(np.arange(inputnodes[1]+1))
# print(vertices)

edges = set(directE)
# print(edges)


f_star = cp.Variable()  # Target flow rate


orientation_constraints = []
flow_constraints = []
equalrate_constraints = []


c =  {a : cp.Variable() for a in edges}

for a in edges:
    orientation_constraints += [c[a] >= 0]      #for every in 2E

for a1 in undirected_edges:         # for every in E
    a2 = (a1[1], a1[0]) 
    orientation_constraints += [ c[a1] + c[a2] == 1]



fia = {(i, a): cp.Variable() for a in edges for i in sessions}
fija = {(i, j, a): cp.Variable() for i in sessions for j in recievers[i] for a in edges}


for i in sessions: 
    for j in recievers[i]:
        for a in edges:
            flow_constraints += [fija[i,j,a] >= 0]
            flow_constraints += [fija[i,j,a] <= fia[i,a]]

for a in edges: 
    session_flow_edge = sum(fia[i,a] for i in sessions)
    flow_constraints += [session_flow_edge <= c[a]]



for i in sessions:
    for j in recievers[i]:
        non_member = vertices - {i} - {j}
        for vt in non_member:
            incoming_edges = [(u, v) for u, v in edges if v == int(vt)] 
            outcoming_edges = [(u, v) for u, v in edges if u == int(vt)] 
            incoming_flow = sum(fija[i, j, x] for x in incoming_edges)
            outcoming_flow = sum(fija[i, j, y] for y in outcoming_edges)
            flow_constraints += [incoming_flow == outcoming_flow]


        incoming_flow_source = sum(fija[i, j, x] for x in edges if x[1] == source[i])  
        
        outcoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[0] == int(j)) 
        incoming_flow_reciever = sum(fija[i, j, x] for x in edges if x[1] == int(j))  

        flow_constraints += [incoming_flow_source == 0]
        flow_constraints += [outcoming_flow_reciever == 0]
        equalrate_constraints += [f_star == incoming_flow_reciever]




objective = cp.Maximize(f_star)    
constraints = orientation_constraints + flow_constraints + equalrate_constraints
problem = cp.Problem(objective, constraints)


problem.solve()

print("Optimal Target Flow Rate (f*):", f_star.value)

Optimal Target Flow Rate (f*): 1.000000000001765
